In [6]:
import glob
import pandas as pd
import json
import datetime
from tqdm import tqdm
import os
from django.conf import settings
#from sqlalchemy import create_engine
os.environ["DJANGO_SETTINGS_MODULE"] = "mysite.settings"
import django
django.setup()
from tqdm import tqdm, tqdm_notebook
import glob
import numpy as np

from django.db.models import Min,Max,Count
from sklearn.neighbors import KDTree
comfig_file_path = '../../main_config.txt'
main_name = 'yandex/'
sub_name = 'raw_org_search/'
import time
from sklearn.utils import shuffle

In [7]:
import sys
import json
sys.path.append('/usr/lib/python3/dist-packages')
from main_map.models import OrganizationNatClass, House, Metro, Distance, City

In [8]:
nat_classes = set(OrganizationNatClass.objects.all().values().values_list(*['nat_class']))

nat_classes = list(map(lambda x:x[0],nat_classes))
nat_classes = shuffle(nat_classes)

In [9]:
#nat_classes = ['АЗС','Парикмахерская', 'Ресторан','Супермаркет']
nat_classes = ['Супермаркет']

In [10]:
lat_min, lon_min = 55.604865, 37.321688
lat_max, lon_max = 55.912290, 37.858644
filter_geo =    {
    'lat__gt' : lat_min, 
    'lon__gt' : lon_min,
    'lat__lt' : lat_max, 
    'lon__lt' : lon_max,
}

In [11]:
# for house in tqdm(House.objects.filter(**filter_geo)):
#     house.nearest_nat_classes_euclid = None
#     house.save()

In [12]:
xy = ['x','y']

In [13]:
# Distance.objects.all().delete()

In [14]:
#Distance.objects.all()[0].__dict__

In [15]:
columns = xy + ['source_name','source_id']
def two_points_distances(objects1,objects2,k_max):
    
    max_distance = 1e10
    df_orgs = pd.DataFrame(list(objects1.values().values_list(*columns)), columns=columns)
    tree = KDTree(df_orgs[xy])
    k = k_max
    #print(len(df_orgs))
    for house in tqdm_notebook(objects2, leave=False):
        #print(house.__dict__)
        distances, indexes = tree.query([[house.x,house.y]], k=k)
        distances = distances[0]
        indexes = indexes[0]
        if distances[0]>max_distance:
            continue
        #print(df_orgs.iloc[indexes])
        distances  = list(np.round(distances,2))
        source_ids = df_orgs['source_id'].iloc[indexes].values
        source_names = df_orgs['source_name'].iloc[indexes].values
        #print(distances)
        for i in range(k):
            obj_to_obj_distance = Distance()

            obj_to_obj_distance.source_id1 = source_ids[i]
            obj_to_obj_distance.source_name1 = source_names[i]
            obj_to_obj_distance.euclid_distance = distances[i]

            obj_to_obj_distance.source_id2 = house.source_id
            obj_to_obj_distance.source_name2 = house.source_name

            obj_to_obj_distance.save()
        

In [11]:
objects2 = House.objects.filter(**filter_geo)

#for table_kmax in tqdm_notebook([(City,1), (Metro,12)], leave=False):


for table_kmax in tqdm_notebook([(Metro,10)], leave=False):
    table, k_max = table_kmax
    objects1 = table.objects.filter(**filter_geo).values().values_list(*columns)
    two_points_distances(objects1,objects2,k_max)

In [ ]:
for nat_class in tqdm_notebook(nat_classes, leave=False):#point 1
    k_max = 40
    filter_dic = filter_geo.copy()
    filter_dic['nat_class'] = nat_class
    objects1 = OrganizationNatClass.objects.filter(**filter_dic)
    two_points_distances(objects1,objects2,k_max)

    

In [ ]:
#for table_kmax in tqdm_notebook([(City,1), (Metro,12)], leave=False):
for table_kmax in tqdm_notebook([(Metro,10)], leave=False):
    table, k_max = table_kmax
    objects1 = table.objects.filter(**filter_geo).values().values_list(*columns)
    for nat_class in tqdm_notebook(nat_classes, leave=False):#point 1
        filter_dic = filter_geo.copy()
        objects2 = OrganizationNatClass.objects.filter(**filter_dic)
        two_points_distances(objects1,objects2,k_max)

In [16]:


unique_fields = ['source_id1', 'source_name1','source_id2', 'source_name2']

duplicates = (
    Distance.objects.values(*unique_fields)
    .order_by()
    .annotate(max_id=Max('id'), count_id=Count('id'))
    .filter(count_id__gt=1)
)
print(len(duplicates))
for duplicate in tqdm_notebook(duplicates):
    (
        Distance.objects
        .filter(**{x: duplicate[x] for x in unique_fields})
        .exclude(id=duplicate['max_id'])
        .delete()
    )

3807706


KeyboardInterrupt: 

In [ ]:
q  =pd.DataFrame(list(Distance.objects.all().values()))

In [ ]:
Distance.objects.all()[0].__dict__

In [ ]:
# [
#     (OrganizationNatClass,'data.mos'),
#     (City,'hand')
# ]


In [ ]:

full_ids = list(OrganizationNatClass.objects.filter(nat_class='Супермаркет').values().values_list(*['source_name','source_id']))
#semi_source_name = 'data.mos'


#full_ids = list(City.objects.values().values_list(*['source_name','source_id']))
# semi_source_name = 'hand'

df_full_ids = pd.DataFrame(full_ids,columns=['source_name','source_id'])
#df_full_ids['source_name'].values

In [ ]:
empty = Distance.objects.filter(
            source_name1 = 'yandex_api',
            source_id1__in = list(df_full_ids['source_id'].values),
            source_name2 = 'reformagkh',
            euclid_distance_metro__isnull = True).count()
notempty = Distance.objects.filter(
            source_name1 = 'yandex_api',
            source_id1__in = list(df_full_ids['source_id'].values),
            source_name2 = 'reformagkh',
            euclid_distance_metro__isnull = False).count()
print('empty',empty)
print('notempty',notempty)

In [5]:
k_max3 = 1

for full_id in tqdm_notebook(
    House.objects.filter(**filter_geo).values().values_list(*['source_name','source_id']),
    leave=False):

    source_name2 = full_id[0]
    source_id2 = full_id[1]
    
    objects = Distance.objects.filter(source_name1='yandex_api',source_id2=source_id2,source_name2=source_name2)
    
    unq = ['source_id1','source_id2','source_name1','source_name2']

    features = unq + ['euclid_distance']

    df1 = pd.DataFrame(list(objects.values().values_list(*features)), columns=features).drop_duplicates(unq)
    
    
    df1 = df1.rename(columns={'source_id2':'source_id3','source_name2':'source_name3'})
    df1 = df1.rename(columns={'source_id1':'source_id2','source_name1':'source_name2'})
    objects = Distance.objects.filter(
        source_id2__in=df1['source_id2'].values,
        source_name2__in=df1['source_name2'].values,
        source_name1='data.mos'
    )
    df2 = pd.DataFrame(list(objects.values().values_list(*features)), columns=features).drop_duplicates(unq)
    df_full = pd.merge(df1,df2,on=['source_id2','source_name2'])
    
    df_full['full_euclid_distance']=  df_full['euclid_distance_x'] + df_full['euclid_distance_y']
    
    #df_full = df_full.drop(['source_name2','source_id2','euclid_distance_x','euclid_distance_y'], axis=1)
    #df_full = df_full.rename(columns={'source_id3':'source_id2','source_name3':'source_name2'})
    
    
    objects = Distance.objects.filter(
        source_name1='data.mos',
        source_name2='reformagkh',

        source_id1__in=df_full['source_id1'].values,
        source_id2__in=df_full['source_id3'].values,
    ).values().values_list(*features)

    df_direct = pd.DataFrame(list(objects),columns=features)
    df_direct = df_direct.rename(columns={'source_id2':'source_id3','source_name2':'source_name3'})
    df_direct = df_direct.rename(columns={'euclid_distance':'direct_euclid_distance'})
    
    df = pd.merge(df_full, df_direct,on=['source_id1', 'source_id3', 'source_name1', 'source_name3'])
    df['euclid_distance_metro'] = df['full_euclid_distance'] - df['direct_euclid_distance']
    df = df.drop(['source_id1','source_name1'], axis=1)
    df = df.rename(columns={'source_id2':'source_id1','source_name2':'source_name1'})
    df = df.rename(columns={'source_id3':'source_id2','source_name3':'source_name2'})
    
    df = df.sort_values('full_euclid_distance')
    
    for i, row in df[unq+['euclid_distance_metro']][:k_max3].iterrows():
        dic = row.to_dict()
        #print(dic)
        euclid_distance_metro = dic.pop('euclid_distance_metro')
        objects = Distance.objects.filter(**dic)
        for obj in objects:
            obj.euclid_distance_metro = euclid_distance_metro
            obj.save()

NameError: name 'filter_geo' is not defined